In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys 
from scipy.linalg import solve

In [3]:
CCOFile = 'stage_2.cco'

In [4]:
def Viscosity(d, hd=0.45):
    mu045 = 220*np.exp(-1.3*d) + 3.2 - 2.44*np.exp(-0.06*d*0.645)
    C = (0.8 + np.exp(-0.075*d))*(-1+(1+10**-11*(d)**12)**-1)+(1+10**-11*(d)**12)**-1
    return 1 + (mu045-1)*((1-hd)**C-1)/((1-0.45)**C-1)

In [5]:
def ReadCCO(CCOFile):
    
    SegmentsData = []
    DistalNodes  = dict()
    
    nodeName = -1

    with open(CCOFile, 'r') as f:
        f.readline()
        f.readline()
        f.readline()
        f.readline()

        nVessels = int(f.readline())

        for i in range(nVessels):
            
            row = (f.readline()).split()
       
            if int(row[-1]) > -2:
                
                nodeName+=1
                DistalNodes[int(row[0])] = nodeName
                
                SegmentsData.append([int(row[0]),
                                     np.array([float(x) for x in row[1:4]])*1e4, 
                                     np.array([float(x) for x in row[4:7]])*1e4,
                                     float(row[12])*1e4,
                                     float(row[13]),
                                     nodeName,
                                     int(row[-1])]) # Id, xProx, xDist, radius, initial flow, distalNode, stage
        
        df = pd.DataFrame(SegmentsData, columns=['Id', 'xProx', 'xDist', 'Radius', 'InitFlow','DistalNode','Stage'])
        df['Inlet'] = False
        df['Outlet'] = False
        df = df.set_index('Id')
        df['ParentId'] = -1
        df['BranchesId'] = [[] for i in range(df.shape[0])]
    
        f.readline()
        f.readline()
        
        NodesConnections = []
        SegNewName = -1
        for i in range(nVessels):
            row = (f.readline()).split()
            SegmentId, ParentId, BranchesIds = int(row[0]), int(row[1]), [int(x) for x in row[2:]]

            if SegmentId in DistalNodes:    
                ProximalNode = DistalNodes[SegmentId]
                branchesId = []
                for connection in BranchesIds:
                    DistalNode = DistalNodes[connection]
                    SegNewName +=1
                    NodesConnections.append((ProximalNode, DistalNode, SegNewName, connection))
                    branchesId.append(connection)
                df.at[SegmentId, 'BranchesId'] = branchesId    
                
                if not BranchesIds:
                    df.at[SegmentId, 'Outlet'] = True
                
                if not ParentId in DistalNodes: # Inlet node, need to add the proximal node to the tree
                    df.at[SegmentId, 'Inlet'] = True
                    nodeName+=1
                    SegNewName +=1
                    NodesConnections.append((nodeName, DistalNodes[SegmentId], SegNewName, SegmentId))
                else:
                    df.at[SegmentId, 'ParentId'] = ParentId

    
     ## Gives each inlet and its downstream branches a number
    def AssignBranchNumberToDownstreamVessels(InletIdx, branchNumber):
        
        # branchNumber = df.at[InletIdx, 'BranchNumber']
        branches = df.at[InletIdx, 'BranchesId']
        
        for branchId in branches:
            branchIdx = df[df.Id==branchId].index[0]
            df.at[branchIdx, 'BranchNumber'] = branchNumber
            AssignBranchNumberToDownstreamVessels(branchIdx, branchNumber)       
        
    df['BranchNumber'] = -1
    for i,row in enumerate(df[df.Inlet].iterrows()):
        df.at[row[0], 'BranchNumber'] = i
        
        AssignBranchNumberToDownstreamVessels(row[0],i)
                    
    
    
    ## Create the matrices for the solver
    ConnectivityMatrix = np.zeros((nodeName+1, len(SegmentsData)))
    Radii  = np.zeros((len(SegmentsData),))
    Length = np.zeros((len(SegmentsData),))
    df['SegName'] = df.index
    df['Id'] = df.index
    D = np.zeros((nodeName+1,nodeName+1)) # Decision matrix

    for proxNode, distNode, SegmentName, SegmentId in NodesConnections:
        
        ConnectivityMatrix[proxNode, SegmentName] = 1
        ConnectivityMatrix[distNode, SegmentName] = -1
        Radii[SegmentName] = df.at[SegmentId, 'Radius']
        xProx, xDist = df.at[SegmentId, 'xProx'], df.at[SegmentId, 'xDist']
        Length[SegmentName] = sum([(xProx[i]-xDist[i])**2 for i in range(3)])**.5
        
        df.at[SegmentId, 'SegName'] = SegmentName
        if df.at[SegmentId, 'Inlet']:
            D[proxNode, proxNode] = 1
        elif df.at[SegmentId, 'Outlet']:
            D[distNode, distNode] = -1
    
    df = df.set_index('SegName')
    

   
        
    return ConnectivityMatrix.T, Radii, Length, D, df

In [6]:
Connectivity, Radii, Length, D, df = ReadCCO(CCOFile)

AttributeError: 'DataFrame' object has no attribute 'Id'

In [7]:
narcs, nnodes = Connectivity.shape
R = np.zeros((narcs,narcs))
for i in range(narcs):
    r,l = Radii[i], Length[i]
    mu  = Viscosity(2*r) 
    R[i,i] = 8.0*mu*l/(np.pi*(r**4))

NameError: name 'Connectivity' is not defined

In [ ]:
# Add the boundary conditions
qperf = 0.3 # muL/min
qcap  = 0.1 # muL/min

# RHS for flow rates: qbar = qperf if inlet node, -qcap otherwise
qbar = np.ones((nnodes,)) # Flow rate at nodes
qbar = np.matmul(D,qbar)
qbar[qbar>0] *= qperf
qbar[qbar<0] *= qcap

pbar = np.ones((nnodes,))
pbar = D.dot(pbar)
pbar[pbar>0] = 25
pbar[pbar<0] = 15

In [ ]:
# Solve the system
print(Connectivity.shape, R.shape, D.shape)
M = np.block([[
    R, -Connectivity],
    [Connectivity.T, D]])
      
RHS = np.concatenate([np.zeros(narcs,), qbar+pbar])
x = solve(M,RHS)
f,p = np.abs(x[:narcs]), x[narcs:]
dp  = np.abs(Connectivity.dot(p))

In [ ]:
df['Flow'] = f
df['PressureDrop'] = dp 

for i in range(f.shape[0]):
    df.at[i,'Flow'] = f[i]
    df.at[i,'PressureDrop'] = dp[i]

fig, axes = plt.subplots(1,3, figsize=(16,4))
for ax in axes:
    ax.set(yscale='log')
sns.scatterplot(data=df, x='Radius', y='Flow', hue='BranchNumber', ax=axes[0])
sns.scatterplot(data=df, x='Radius', y='InitFlow', hue='BranchNumber', ax=axes[1])
sns.scatterplot(data=df, x='Radius', y='PressureDrop' , hue='BranchNumber', ax=axes[2])

In [ ]:
Tot = 0
for i in range(10):
    termCap = df[(df.Outlet) & (df.BranchNumber==i)].shape[0]
    Tot +=termCap
    print(f'Branch {i} has {termCap} terminal capillaries.')
print(f'For a total of {Tot} terminal capillaries, where it should be {df[df.Outlet].shape[0]}.')